In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jul 23 12:58:28 2019

@author: fame
""" 
import os  
import torch
import numpy as np
import os.path 
 
 
def _isArrayLike(obj):
    return hasattr(obj, '__iter__') and hasattr(obj, '__len__')

 
def load_data(split_load, actions_dict, GT_folder, DATA_folder, features_folder, datatype = 'training'):
    file_ptr = open(split_load, 'r')
    content_all = file_ptr.read().split('\n')[1:-1]
    content_all = [x.strip('./data/groundTruth/') + 't' for x in content_all]
    all_tasks = ['tea', 'cereals', 'coffee', 'friedegg', 'juice', 'milk', 'sandwich', 'scrambledegg', 'pancake', 'salat']
    
    if datatype == 'training':
        data_breakfast = []
        labels_breakfast = []
        for content in content_all:
        
            file_ptr = open( GT_folder + content, 'r')
            curr_gt = file_ptr.read().split('\n')[:-1]
            loc_curr_data = DATA_folder + os.path.splitext(content)[0] + '.gz'
            curr_data = np.loadtxt(loc_curr_data, dtype='float32')
            feature_name = features_folder + os.path.splitext(content)[0] + '.npy'
            np.save(feature_name, curr_data)
            label_curr_video = []
            for iik in range(len(curr_gt)):
                label_curr_video.append( actions_dict[curr_gt[iik]] )

            data_breakfast.append(torch.tensor(curr_data,  dtype=torch.float64 ) )
            labels_breakfast.append(label_curr_video )
    
        labels_uniq, labels_uniq_loc = get_label_bounds(labels_breakfast)
        print("Finish Load the Training data and labels!!!")     
        return  data_breakfast, labels_uniq
    if datatype == 'test':
        data_breakfast = []
        
        segment = []
        for content in content_all:
            
            #file_ptr = open( GT_folder + content, 'r')
            #curr_gt = file_ptr.read().split('\n')[:-1]
            
            loc_curr_data = DATA_folder + os.path.splitext(content)[0] + '.gz'
        
            curr_data = np.loadtxt(loc_curr_data, dtype='float32')
            feature_name = features_folder + os.path.splitext(content)[0] + '.npy'
            np.save(feature_name, curr_data)
                                 
            data_breakfast.append(torch.tensor(curr_data,  dtype=torch.float64 ) )
            
        
        return data_breakfast


def get_label_bounds( data_labels):
    labels_uniq = []
    labels_uniq_loc = []
    for kki in range(0, len(data_labels) ):
        uniq_group, indc_group = get_label_length_seq(data_labels[kki])
        labels_uniq.append(uniq_group[1:-1])
        labels_uniq_loc.append(indc_group[1:-1])
    return labels_uniq, labels_uniq_loc

def get_label_length_seq(content):
    label_seq = []
    length_seq = []
    start = 0
    length_seq.append(0)
    for i in range(len(content)):
        if content[i] != content[start]:
            #print(content[i])
            label_seq.append(content[start])
            length_seq.append(i)
            start = i
    label_seq.append(content[start])
    length_seq.append(len(content))
    
    if content[-1] != 0:
        label_seq.append(content[-1])
    
    return label_seq, length_seq


def get_maxpool_lstm_data(cData, indices):
    list_data = []
    for kkl in range(len(indices)-1):
        cur_start = indices[kkl]
        cur_end = indices[kkl+1]
        if cur_end > cur_start:
            list_data.append(torch.max(cData[cur_start:cur_end,:],
                                       0)[0].squeeze(0))
        else:
            list_data.append(torch.max(cData[cur_start:cur_end+1,:],
                                       0)[0].squeeze(0))
    list_data  =  torch.stack(list_data)
    return list_data

def read_mapping_dict(mapping_file):
    file_ptr = open(mapping_file, 'r')
    actions = file_ptr.read().split('\n')[:-1]

    actions_dict=dict()
    for a in actions:
        actions_dict[a.split()[1]] = int(a.split()[0])

    return actions_dict

if __name__ == "__main__":
    COMP_PATH = ''
    split = 'training'
#     split = 'test'
    train_split =  os.path.join(COMP_PATH, 'splits/train.split1.bundle')
    test_split  =  os.path.join(COMP_PATH, 'splits/test.split1.bundle')
    GT_folder   =  os.path.join(COMP_PATH, 'groundTruth/')
    DATA_folder =  os.path.join(COMP_PATH, 'data/')
    mapping_loc =  os.path.join(COMP_PATH, 'splits/mapping_bf.txt')
    train_features_folder = os.path.join(COMP_PATH, 'train_features/')
    test_features_folder = os.path.join(COMP_PATH, 'test_features/')

    
    
  
    actions_dict = read_mapping_dict(mapping_loc)
    if  split == 'training':
        data_feat, data_labels = load_data(train_split, actions_dict, GT_folder, DATA_folder, train_features_folder, datatype = split)
        
    if  split == 'test':
        data_feat = load_data(test_split, actions_dict, GT_folder, DATA_folder, test_features_folder, datatype = split)
    
 




Finish Load the Training data and labels!!!


In [ ]:
data_feat[0]

In [12]:
import numpy as np
import os

In [8]:
predicted = np.random.randint(48, size=(252, 8500))

In [9]:
predicted.shape


(252, 8500)

In [10]:
predicted

array([[12, 15, 26, ...,  0,  2, 27],
       [12,  2, 11, ..., 25, 25, 29],
       [32, 10, 25, ..., 39, 33, 11],
       ...,
       [46, 32, 38, ..., 30, 39, 35],
       [47, 47, 23, ..., 44, 23, 12],
       [ 0, 35, 25, ..., 27, 24, 44]])

In [13]:
def read_mapping_dict(mapping_file):
    file_ptr = open(mapping_file, 'r')
    actions = file_ptr.read().split('\n')[:-1]

    actions_dict=dict()
    for a in actions:
        actions_dict[a.split()[1]] = int(a.split()[0])

    return actions_dict

In [16]:
COMP_PATH = ''
mapping_file = os.path.join(COMP_PATH, "data/splits/mapping_bf.txt")
file_ptr = open(mapping_file, 'r')
actions = file_ptr.read().split('\n')[:-1]
file_ptr.close()
actions_dict = dict()
for a in actions:
    actions_dict[a.split()[1]] = int(a.split()[0])


In [17]:
actions_dict

{'SIL': 0,
 'add_saltnpepper': 12,
 'add_teabag': 45,
 'butter_pan': 17,
 'crack_egg': 11,
 'cut_bun': 36,
 'cut_fruit': 32,
 'cut_orange': 19,
 'fry_egg': 13,
 'fry_pancake': 29,
 'peel_fruit': 34,
 'pour_cereals': 1,
 'pour_coffee': 5,
 'pour_dough2pan': 28,
 'pour_egg2pan': 43,
 'pour_flour': 31,
 'pour_juice': 21,
 'pour_milk': 2,
 'pour_oil': 10,
 'pour_sugar': 9,
 'pour_water': 46,
 'put_bunTogether': 40,
 'put_egg2plate': 15,
 'put_fruit2bowl': 33,
 'put_pancake2plate': 30,
 'put_toppingOnTop': 39,
 'smear_butter': 37,
 'spoon_flour': 26,
 'spoon_powder': 24,
 'spoon_sugar': 7,
 'squeeze_orange': 20,
 'stir_cereals': 3,
 'stir_coffee': 8,
 'stir_dough': 27,
 'stir_egg': 42,
 'stir_fruit': 35,
 'stir_milk': 25,
 'stir_tea': 47,
 'stirfry_egg': 44,
 'take_bowl': 4,
 'take_butter': 41,
 'take_cup': 6,
 'take_eggs': 16,
 'take_glass': 22,
 'take_knife': 18,
 'take_plate': 14,
 'take_squeezer': 23,
 'take_topping': 38}

In [20]:

test_segment =  os.path.join(COMP_PATH, 'test_segment.txt')
recognition = []
for i in range(len(predicted)):
    recognition = np.concatenate((recognition, [actions_dict.keys()[actions_dict.values().index(predicted[i].item())]]))


AttributeError: 'dict_values' object has no attribute 'index'

In [ ]:
f_name = vid.split('/')[-1].split('.')[0]
f_ptr = open(results_dir + "/" + f_name, "w")
f_ptr.write("### Frame level recognition: ###\n")
f_ptr.write(' '.join(recognition))
f_ptr.close()